In [52]:
import numpy as np
import pandas as pd
import bjorn_support as bs
import plotly.graph_objects as go

In [77]:
afp = '/home/gk/southpark'
bs.get_filepaths(afp, data_fmt='fastq.gz', data_type='')

{}

In [39]:
analysis_fp = '/home/al/analysis/11-16.11.2020'

In [49]:
def get_amplicon_filter_data(analysis_filepath):
    log_fps = bs.get_filepaths(analysis_filepath, data_fmt='log', data_type='logs', tech='')
    amplicon_filter_data = {}
    for idx, fp, in log_fps.items():
        with open(fp) as f:
            lines = f.readlines()
        filtered_reads = get_num(lines[-2])
        total_reads = get_total(lines[-5])
        amplicon_filter_data[idx] = (filtered_reads, total_reads, filtered_reads/total_reads)
    return amplicon_filter_data

def get_total(line):
    start_idx = line.find('from')+5
    end_idx = line.find('%')
    pct = line[start_idx:end_idx]
    start_idx = line.find('(')+1
    end_idx = line.find(')')
    cnt = line[start_idx:end_idx]
    return int(eval(cnt)*100 / eval(pct))

def get_num(line):
    end_idx  = line.find('reads')
    return eval(line[:end_idx])

afd = get_amplicon_filter_data(analysis_fp)
# afd

In [50]:
afd = (pd.DataFrame(data=afd.values(), index=afd.keys(), 
                   columns=['filtered_reads', 'total_reads', 'pct_filtered'])
       .reset_index().rename(columns={'index': 'idx'}))
afd

,idx,filtered_reads,total_reads,pct_filtered
0,4371,385448,5135120,0.075061
1,4409,357181,4611738,0.077450
2,4105,8937,13265,0.673728
3,4328,20359,351478,0.057924
4,4195,1246604,2472622,0.504163
...,...,...,...,...
382,4406,14831,461973,0.032104
383,4354,20407,32662,0.624793
384,4447,689332,4045890,0.170378
385,4277,12065,56187,0.214729


In [75]:
fig = go.Figure(go.Box(y=afd['pct_filtered'], name='11-16.11.2020', 
                       text=afd[['idx', 'filtered_reads', 'total_reads', 'pct_filtered']], 
                       boxpoints='all', jitter=0.3,
                       hovertemplate = "<b>%{text[0]}</b><br>" +
                                      "<b>Reads filtered: %{text[1]}</b><br>" +
                                      "<b>Total reads: %{text[2]}</b><br>" +
                                      "<b>% filtered: %{text[3]:.2f}</b><br>"
                      )
               )
fig.update_layout(yaxis_title='% reads filtered', xaxis_title='amplicon filter step (iVar1.3)',
                  template='plotly_white', autosize=False, height=850, width=800)
fig.write_html('test_data/amplicon_filter_boxplot.html')
fig.show()